In [1]:
from datasketch import MinHash,MinHashLSH,MinHashLSHForest

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import jieba.posseg as pseg
import re

In [4]:
f = open('E:\AI工程师计算广告学习\Action\L8\weibos.txt','r',encoding='UTF-8')
text = f.read()

In [5]:
#将句号、叹号、问好作为分隔，去掉/n换行
sentences = re.split('[。！？]',text.replace('\n',''))

In [6]:
if sentences[len(sentences)-1] == '':
    sentences.pop()

In [7]:
def get_item_str(item_text):
    item_str = ""
    item = (pseg.cut(item_text))
    for i in list(item):
        if i.word not in list(stop):
            item_str += i.word
            #tfidf_vectorizer.fit_transform的输入需要空格分割的单词
            item_str += ' '
    return item_str
#对item——str创建minHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp 

In [8]:
stop  = [line.strip().decode('utf-8') for line in open('stopword.txt').readlines()]

documents = []
for item_text in sentences:
    item_str = get_item_str(item_text)
    documents.append(item_str)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\邢开源\AppData\Local\Temp\jieba.cache
Loading model cost 1.671 seconds.
Prefix dict has been built succesfully.


In [9]:
# 创建LSH Forest及MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)
# index所有key，以便可以进行检索
forest.index()

In [18]:
query = '国足输给叙利亚'
# 将item_text进行分词
item_str = get_item_str(query)
# 得到item_str的MinHash
minhash_query = get_minhash(item_str)

In [19]:
# 查询forest中与m1相似的Top-K个邻居
result = forest.query(minhash_query, 5)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))
print("Top 3 邻居", result)

1 0.5234375 ​国足输给叙利亚之后，里皮辞职
35 0.1328125 男足的水平也就跟南极洲企鹅踢球
5 0.125 据了解
37 0.3359375 国足昨晚1-2输给叙利亚，赛后主帅里皮宣布辞职
22 0.0546875 ”，球员委屈的说：“七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛
Top 3 邻居 [1, 35, 5, 37, 22]
